In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Air Quality Prediction - Evaluation\n",
    "\n",
    "## Tujuan\n",
    "Notebook ini bertujuan untuk mengevaluasi model terbaik di data test dan melakukan analisis lebih mendalam.\n",
    "\n",
    "## Langkah-langkah:\n",
    "1. Load model terbaik\n",
    "2. Prediksi di data test\n",
    "3. Hitung metrik evaluasi\n",
    "4. Analisis residual\n",
    "5. Interpretasi model (feature importance)\n",
    "6. Simpan hasil evaluasi"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Import libraries\n",
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "import joblib\n",
    "from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error\n",
    "from sklearn.inspection import permutation_importance\n",
    "import shap\n",
    "import warnings\n",
    "\n",
    "warnings.filterwarnings('ignore')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Load data test\n",
    "X_test = pd.read_csv('data/processed/X_test.csv', index_col='date', parse_dates=True)\n",
    "y_test = pd.read_csv('data/processed/y_test.csv', index_col='date', parse_dates=True).squeeze()\n",
    "\n",
    "# Load model terbaik\n",
    "best_model = joblib.load('models/trained_models/xgboost_model.pkl')\n",
    "print(\"Best model loaded.\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Prediksi di data test\n",
    "y_pred = best_model.predict(X_test)\n",
    "\n",
    "# Hitung metrik evaluasi\n",
    "mse = mean_squared_error(y_test, y_pred)\n",
    "rmse = np.sqrt(mse)\n",
    "mae = mean_absolute_error(y_test, y_pred)\n",
    "r2 = r2_score(y_test, y_pred)\n",
    "\n",
    "print(\"Final Evaluation Metrics:\")\n",
    "print(f\"RMSE: {rmse:.4f}\")\n",
    "print(f\"MAE: {mae:.4f}\")\n",
    "print(f\"R2: {r2:.4f}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Analisis residual\n",
    "residuals = y_test - y_pred\n",
    "\n",
    "plt.figure(figsize=(12, 6))\n",
    "plt.subplot(1, 2, 1)\n",
    "sns.histplot(residuals, kde=True)\n",
    "plt.title('Distribution of Residuals')\n",
    "plt.xlabel('Residuals')\n",
    "\n",
    "plt.subplot(1, 2, 2)\n",
    "sns.scatterplot(x=y_pred, y=residuals)\n",
    "plt.axhline(y=0, color='r', linestyle='--')\n",
    "plt.title('Residuals vs Predicted Values')\n",
    "plt.xlabel('Predicted AQI')\n",
    "plt.ylabel('Residuals')\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Feature importance\n",
    "feature_importances = pd.Series(best_model.feature_importances_, index=X_test.columns)\n",
    "top_features = feature_importances.sort_values(ascending=False).head(10)\n",
    "\n",
    "plt.figure(figsize=(10, 6))\n",
    "top_features.sort_values().plot(kind='barh')\n",
    "plt.title('Top 10 Feature Importances')\n",
    "plt.xlabel('Importance Score')\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# SHAP values for model interpretation\n",
    "# Karena mungkin memakan waktu, kita gunakan subset data\n",
    "X_sample = X_test.sample(100, random_state=42)\n",
    "\n",
    "# Initialize JS for visualization\n",
    "shap.initjs()\n",
    "\n",
    "# Create explainer\n",
    "explainer = shap.TreeExplainer(best_model)\n",
    "shap_values = explainer.shap_values(X_sample)\n",
    "\n",
    "# Summary plot\n",
    "plt.figure()\n",
    "shap.summary_plot(shap_values, X_sample, plot_type=\"bar\")\n",
    "plt.title('SHAP Feature Importance')\n",
    "plt.show()\n",
    "\n",
    "# SHAP summary plot (beeswarm)\n",
    "plt.figure()\n",
    "shap.summary_plot(shap_values, X_sample)\n",
    "plt.title('SHAP Summary Plot')\n",
    "plt.show()\n",
    "\n",
    "# Contoh force plot untuk satu observasi\n",
    "print(\"SHAP Force Plot for First Observation\")\n",
    "shap.force_plot(explainer.expected_value, shap_values[0,:], X_sample.iloc[0,:])"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Simpan hasil evaluasi\n",
    "eval_results = {\n",
    "    'RMSE': rmse,\n",
    "    'MAE': mae,\n",
    "    'R2': r2\n",
    "}\n",
    "\n",
    "eval_df = pd.DataFrame(eval_results, index=['Value'])\n",
    "eval_df.to_csv('reports/evaluation_metrics.csv')\n",
    "print(\"Evaluation metrics saved to 'reports/evaluation_metrics.csv'\")"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.5"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}